Using the celeb faces data set  pick 5 people and train classifiers to recogtnise their faces using no augmentation, or different types of augmentation. 
Questions to investigate:
1. Which of these are valid transformations for human faces?:
 - horizontal shifts
 - vertical shifts
 - rotation
 - horizontal flips
 - vertical flips
To do this investigation using appropriate scientific method, treat each of these as a hypothesis to be tested. Take a number of observations (e.g. accuracy of trained model) for each case (e.g. using horizontal flips vs not using horizontal flips) then compare the meana results and use appropriate statistcal tests to determine whethe the results are statistically significanrtly differernt.

### class discussion
does data augmentation provide a away of addressing:
- ethical concerns about under-representation of certain groups
- safety concerns for example wrt autonomous vehicles